In [96]:
import cv2
import numpy as np
import os
from itertools import chain
import pandas as pd

def image_index_cal(directory):
    '''
    Calculates the image quality index for all images in the given directory
    and saves the results as a CSV file.

    Args:
        directory (str): The directory containing the images to process.

    Returns:
        None
    '''
    # Ensure that the input directory is valid
    if not os.path.isdir(directory):
        raise ValueError("Input directory is not a valid directory")

    # Get the list of files in the directory
    files = os.listdir(directory)
    
    image_list = []
    
    # Filter the list to only include image files
    for file in files:
        if file.endswith(('.png','.bmp','.jpg', 'JPEG')):
            image_list.append(file)
    # Raise an error if no images were found in the directory
    if not image_list:
        raise ValueError("Input directory does not contain any images")
            
    QI_list = []
    
    # Loop over each image in the directory
    for index in range(len(image_list)):
        
        # Read the image and convert it to a 1D vector
        img_arr = cv2.imread(os.path.join(directory, image_list[index]), 0)
        vec_1D = list(chain.from_iterable(img_arr))
        
        # Calculate the image quality index based on percentiles of pixel values
        low = np.percentile(vec_1D, 1)
        satu = np.percentile(vec_1D, 99)
        noise = np.percentile(vec_1D, 30)
        middle = np.mean([noise, satu])
        
        # Replace 0 with 1 to avoid division by zero
        if low < 1:
            low = 1
        #Calculate intensity ratio    
        IR = (satu - low) / low * 100
        
        # Count the number of pixels in the noise/middle and saturation ranges
        NM_pixels = len(list(x for x in vec_1D if noise <= x <= middle))
        MS_pixels = len(list(y for y in vec_1D if middle < y <= satu))
        
        TSR = MS_pixels / NM_pixels
        
        # Calculate the image quality index (QI)
        QI = IR*TSR
        
        # Append the QI to the list of results
        QI_list.append(QI)
    
    # Create a pandas dataframe to store the results
    column_dic = {'FILE':image_list,'quality_index':QI_list}
    df_name_qi = pd.DataFrame(column_dic, columns =['FILE', 'quality_index'])
    
    # Save the results as a CSV file in the same directory
    df_name_qi.to_csv(os.path.join(directory, "Image_quality_table.csv"), index=False)
    
    return 
